In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

In [9]:
df_final = pd.read_table("df_prognosis_features_ready.tsv")
#x = df_final.iloc[:,1:177]
x = df_final.iloc[:,1:84]
y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [33]:
{'alpha': 10. ** np.arange(-6, -4, 0.1)}

{'alpha': array([1.00000000e-06, 1.25892541e-06, 1.58489319e-06, 1.99526231e-06,
        2.51188643e-06, 3.16227766e-06, 3.98107171e-06, 5.01187234e-06,
        6.30957344e-06, 7.94328235e-06, 1.00000000e-05, 1.25892541e-05,
        1.58489319e-05, 1.99526231e-05, 2.51188643e-05, 3.16227766e-05,
        3.98107171e-05, 5.01187234e-05, 6.30957344e-05, 7.94328235e-05])}

In [34]:
estimator = FastSurvivalSVM(optimizer="rbtree", max_iter=1000, tol=1e-6, random_state=17)
#estimator = FastKernelSurvivalSVM(optimizer="rbtree", kernel="precomputed", random_state=0)

In [35]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [44]:
param_grid = {'alpha': 10. ** np.arange(-6, -4, 0.1)}
#param_grid = {'alpha': 10. ** np.arange(-10, -3, 0.2)}
#cv = ShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
cv = ShuffleSplit(n_splits=10,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=4, iid=False, refit=False,
                   cv=cv)

In [45]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
gcv = gcv.fit(x,y)
#gcv = gcv.fit(clinical_kernel(x),y)


In [46]:
gcv.best_score_

0.6464302951335703

In [47]:
gcv.best_params_

{'alpha': 7.943282347242757e-06}